In [80]:
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D, Flatten, Dense
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras import backend as K
from keras import initializers
import keras

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

In [2]:
img_width, img_height = 31, 31

nb_epoch = 50
batch_size = 16

In [3]:
base_dir = '/scratch/image_datasets/1_for_learned_sift/ridiculously_small'

train_data_dir      = base_dir + '/patches/train'
validation_data_dir = base_dir + '/patches/validation'
# test_data_dir       = base_dir + '/patches/test'

train_descrs_dir      = base_dir + '/descriptors_angles/train'
validation_descrs_dir = base_dir + '/descriptors_angles/validation'
# test_descrs_dir       = base_dir + '/descriptors_angles/test'

In [4]:
def loading_data(dir_patches, dir_descrs):
    files_patches = listdir(dir_patches + '/class0')
    files_patches.sort()
    
    files_descrs = listdir(dir_descrs + '/class0')
    files_descrs.sort()
    
    assert len(files_patches) == len(files_descrs), "The number of patches doesn't match the number of descriptors."

    patches = []
    descrs = []

    
    for file_patch, file_descr in zip(files_patches, files_descrs):
        patch = imageio.imread(dir_patches + '/class0/' + file_patch)
#         print(patch.shape)
        if patch.shape[0] == 31:
            patches.append(patch)
            descr = np.load(dir_descrs + '/class0/' + file_descr)
            descrs.append(descr)
#         elif image.shape[0] == 19:
#             temp_count19 += 1
        
        

    patches = np.array(patches)
    patches = patches.astype(np.float64) / 255
    
    descrs = np.array(descrs)
    descrs = (descrs.astype(np.float64) / 255)
    print("patches", patches.shape, "  descrs", descrs.shape)
    
    return patches, descrs

In [5]:
x_train, y_train = loading_data(train_data_dir, train_descrs_dir)
x_validation, y_validation = loading_data(validation_data_dir, validation_descrs_dir)

patches (35, 31, 31)   descrs (35, 129)
patches (25, 31, 31)   descrs (25, 129)


In [6]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[2], x_train.shape[2], 1))
x_validation = x_validation.reshape((x_validation.shape[0], x_validation.shape[2], x_validation.shape[2], 1))

In [7]:
x_validation.shape

(25, 31, 31, 1)

In [26]:
def MSLE_plus_plus(y_true, y_pred):
    if not K.is_tensor(y_pred):
        y_pred = K.constant(y_pred)
    y_true = K.cast(y_true, y_pred.dtype)
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
    custom_loss_log = K.log(K.clip(y_true + y_pred, K.epsilon(), None) + 1.)
    custom_loss_denominator = (y_true * y_pred + 0.005) * 256  # parameters to be further adjusted
    return K.mean(K.square(first_log - second_log) + custom_loss_log / custom_loss_denominator, axis=-1)

In [10]:
def fixed_generator(x_train, y_train, batch_size):
    while True:
        batch_list_x = []
        batch_list_y = []
        
        for i in range(x_train.shape[0]):
            batch_list_x.append(x_train[i])
            batch_list_y.append(y_train[i])
            if len(batch_list_x) == batch_size:
                yield (np.array(batch_list_x),np.array(batch_list_y))
                batch_list_x = []
                batch_list_y = []


In [8]:
y_validation[0]

array([0.01960784, 0.03137255, 0.03137255, 0.01176471, 0.05490196,
       0.02745098, 0.09411765, 0.18039216, 0.03137255, 0.05490196,
       0.10196078, 0.02352941, 0.05490196, 0.20392157, 0.2745098 ,
       0.13333333, 0.00784314, 0.00392157, 0.03137255, 0.05490196,
       0.36470588, 0.49019608, 0.09019608, 0.04705882, 0.02745098,
       0.00392157, 0.00392157, 0.00392157, 0.03529412, 0.09411765,
       0.08235294, 0.17647059, 0.01176471, 0.17254902, 0.49019608,
       0.05882353, 0.00392157, 0.        , 0.00392157, 0.01960784,
       0.06666667, 0.1254902 , 0.49019608, 0.21176471, 0.02745098,
       0.01176471, 0.00392157, 0.01568627, 0.23529412, 0.0745098 ,
       0.25098039, 0.11764706, 0.18823529, 0.09411765, 0.23529412,
       0.28627451, 0.01176471, 0.        , 0.        , 0.        ,
       0.03137255, 0.30196078, 0.49019608, 0.18823529, 0.        ,
       0.09803922, 0.45098039, 0.10980392, 0.        , 0.        ,
       0.        , 0.        , 0.2627451 , 0.22745098, 0.38039

In [11]:
input_shape = (img_width, img_height, 1)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="elu", padding="valid")(input_img)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(16, (3, 3), activation="elu", padding="valid")(x)
x = MaxPooling2D((4, 4), padding="valid")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(129, activation="sigmoid")(x)

encoder1 = Model(input_img, encoded)

encoder1.compile(optimizer='adadelta', loss="binary_crossentropy")
encoder1.summary()

model_version = '2.0.0.0.8_encoder_elueverywhere_sigmoidlast_bce_adadelta_ridiculouslysmall'

checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder1.fit_generator(fixed_generator(x_train, y_train, batch_size),
                steps_per_epoch=x_train.shape[0],
                epochs=500,
                validation_data=fixed_generator(x_validation, y_validation, batch_size),
                validation_steps=x_validation.shape[0]#,
#                 callbacks=[checkpointer]
                )
encoder1.save(base_dir + '/ae' + model_version + '.h5')


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 31, 31, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 32)        9248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 23, 23, 32)        9248      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 16)          0   

Epoch 67/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2499 - val_loss: 0.4911
Epoch 68/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2512 - val_loss: 0.4965
Epoch 69/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2498 - val_loss: 0.4977
Epoch 70/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2512 - val_loss: 0.4997
Epoch 71/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2497 - val_loss: 0.4916
Epoch 72/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2510 - val_loss: 0.4960
Epoch 73/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2496 - val_loss: 0.4974
Epoch 74/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2507 - val_loss: 0.4944
Epoch 75/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2497 - val_loss: 0.5042
Epoch 76/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2509 - val_los

35/35 [==============================] - 1s 16ms/step - loss: 0.2486 - val_loss: 0.5225
Epoch 148/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2499 - val_loss: 0.5221
Epoch 149/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2486 - val_loss: 0.5258
Epoch 150/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2498 - val_loss: 0.5257
Epoch 151/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2486 - val_loss: 0.5285
Epoch 152/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2499 - val_loss: 0.5291
Epoch 153/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2485 - val_loss: 0.5288
Epoch 154/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2498 - val_loss: 0.5313
Epoch 155/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2486 - val_loss: 0.5247
Epoch 156/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2499 - val_loss: 0

35/35 [==============================] - 1s 15ms/step - loss: 0.2483 - val_loss: 0.5405
Epoch 228/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2496 - val_loss: 0.5391
Epoch 229/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2483 - val_loss: 0.5420
Epoch 230/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2496 - val_loss: 0.5434
Epoch 231/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2483 - val_loss: 0.5453
Epoch 232/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2496 - val_loss: 0.5470
Epoch 233/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2483 - val_loss: 0.5445
Epoch 234/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2496 - val_loss: 0.5434
Epoch 235/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2483 - val_loss: 0.5435
Epoch 236/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2495 - val_loss: 0

35/35 [==============================] - 1s 16ms/step - loss: 0.2482 - val_loss: 0.5537
Epoch 308/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2495 - val_loss: 0.5549
Epoch 309/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2482 - val_loss: 0.5583
Epoch 310/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2495 - val_loss: 0.5585
Epoch 311/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2482 - val_loss: 0.5573
Epoch 312/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2495 - val_loss: 0.5566
Epoch 313/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2482 - val_loss: 0.5568
Epoch 314/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2495 - val_loss: 0.5554
Epoch 315/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2482 - val_loss: 0.5573
Epoch 316/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2494 - val_loss: 0

35/35 [==============================] - 1s 16ms/step - loss: 0.2481 - val_loss: 0.5672
Epoch 388/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2494 - val_loss: 0.5694
Epoch 389/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2481 - val_loss: 0.5696
Epoch 390/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2494 - val_loss: 0.5655
Epoch 391/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2481 - val_loss: 0.5676
Epoch 392/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2494 - val_loss: 0.5706
Epoch 393/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2481 - val_loss: 0.5686
Epoch 394/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2494 - val_loss: 0.5667
Epoch 395/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2481 - val_loss: 0.5662
Epoch 396/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2494 - val_loss: 0

35/35 [==============================] - 1s 16ms/step - loss: 0.2481 - val_loss: 0.5760
Epoch 468/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2493 - val_loss: 0.5748
Epoch 469/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2481 - val_loss: 0.5777
Epoch 470/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2493 - val_loss: 0.5773
Epoch 471/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2481 - val_loss: 0.5756
Epoch 472/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2494 - val_loss: 0.5785
Epoch 473/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2481 - val_loss: 0.5799
Epoch 474/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2494 - val_loss: 0.5763
Epoch 475/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2481 - val_loss: 0.5764
Epoch 476/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2494 - val_loss: 0

In [12]:
input_shape = (img_width, img_height, 1)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="elu", padding="valid")(input_img)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(16, (3, 3), activation="elu", padding="valid")(x)
x = MaxPooling2D((4, 4), padding="valid")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(129, activation="elu")(x)

encoder2 = Model(input_img, encoded)

encoder2.compile(optimizer='adadelta', loss="binary_crossentropy")
encoder2.summary()

model_version = '2.0.0.0.9_encoder_elueverywhere_bce_adadelta_ridiculouslysmall'

checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder2.fit_generator(fixed_generator(x_train, y_train, batch_size),
                steps_per_epoch=x_train.shape[0],
                epochs=500,
                validation_data=fixed_generator(x_validation, y_validation, batch_size),
                validation_steps=x_validation.shape[0]#,
#                 callbacks=[checkpointer]
                )
encoder2.save(base_dir + '/ae' + model_version + '.h5')


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 31, 31, 1)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 27, 27, 32)        9248      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 25, 25, 32)        9248      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 23, 23, 32)        9248      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 16)          0   

Epoch 67/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2824 - val_loss: 0.5713
Epoch 68/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2846 - val_loss: 0.5756
Epoch 69/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2767 - val_loss: 0.5765
Epoch 70/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2744 - val_loss: 0.5791
Epoch 71/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2728 - val_loss: 0.5655
Epoch 72/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2758 - val_loss: 0.5813
Epoch 73/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2718 - val_loss: 0.5839
Epoch 74/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2766 - val_loss: 0.5778
Epoch 75/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2709 - val_loss: 0.5973
Epoch 76/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2724 - val_los

35/35 [==============================] - 1s 16ms/step - loss: 0.2693 - val_loss: 0.5749
Epoch 148/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2700 - val_loss: 0.5914
Epoch 149/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2664 - val_loss: 0.5958
Epoch 150/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2721 - val_loss: 0.5826
Epoch 151/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2672 - val_loss: 0.5735
Epoch 152/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2729 - val_loss: 0.5818
Epoch 153/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2689 - val_loss: 0.5824
Epoch 154/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2687 - val_loss: 0.5821
Epoch 155/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2701 - val_loss: 0.5661
Epoch 156/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2703 - val_loss: 0

35/35 [==============================] - 1s 16ms/step - loss: 0.2699 - val_loss: 0.5874
Epoch 228/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2689 - val_loss: 0.5886
Epoch 229/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2677 - val_loss: 0.5879
Epoch 230/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2689 - val_loss: 0.5800
Epoch 231/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2674 - val_loss: 0.5738
Epoch 232/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2699 - val_loss: 0.5787
Epoch 233/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2662 - val_loss: 0.5800
Epoch 234/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2675 - val_loss: 0.5789
Epoch 235/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2677 - val_loss: 0.5698
Epoch 236/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2701 - val_loss: 0

35/35 [==============================] - 1s 16ms/step - loss: 0.2661 - val_loss: 0.5751
Epoch 308/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2700 - val_loss: 0.5373
Epoch 309/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2684 - val_loss: 0.5707
Epoch 310/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2670 - val_loss: 0.5595
Epoch 311/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2697 - val_loss: 0.5663
Epoch 312/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2691 - val_loss: 0.5672
Epoch 313/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2657 - val_loss: 0.5779
Epoch 314/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2717 - val_loss: 0.5718
Epoch 315/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2668 - val_loss: 0.5696
Epoch 316/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2695 - val_loss: 0

35/35 [==============================] - 1s 16ms/step - loss: 0.2672 - val_loss: 0.5705
Epoch 388/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2679 - val_loss: 0.5599
Epoch 389/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2663 - val_loss: 0.5756
Epoch 390/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2676 - val_loss: 0.5373
Epoch 391/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2659 - val_loss: 0.5568
Epoch 392/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2682 - val_loss: 0.5619
Epoch 393/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2676 - val_loss: 0.5368
Epoch 394/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2700 - val_loss: 0.5456
Epoch 395/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2663 - val_loss: 0.5677
Epoch 396/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2671 - val_loss: 0

35/35 [==============================] - 1s 16ms/step - loss: 0.2694 - val_loss: 0.5590
Epoch 468/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2693 - val_loss: 0.5712
Epoch 469/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2692 - val_loss: 0.5647
Epoch 470/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2679 - val_loss: 0.5776
Epoch 471/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2656 - val_loss: 0.5692
Epoch 472/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2673 - val_loss: 0.5708
Epoch 473/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2690 - val_loss: 0.5621
Epoch 474/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2693 - val_loss: 0.5328
Epoch 475/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2675 - val_loss: 0.5573
Epoch 476/500
35/35 [==============================] - 1s 17ms/step - loss: 0.2687 - val_loss: 0

In [13]:
input_shape = (img_width, img_height, 1)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="elu", padding="valid")(input_img)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(16, (3, 3), activation="elu", padding="valid")(x)
x = MaxPooling2D((4, 4), padding="valid")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(129, activation="elu")(x)

encoder3 = Model(input_img, encoded)

encoder3.compile(optimizer='adadelta', loss="mse")
encoder3.summary()

model_version = '2.0.0.0.10_encoder_elueverywhere_mse_adadelta_ridiculouslysmall'

checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder3.fit_generator(fixed_generator(x_train, y_train, batch_size),
                steps_per_epoch=x_train.shape[0],
                epochs=500,
                validation_data=fixed_generator(x_validation, y_validation, batch_size),
                validation_steps=x_validation.shape[0]#,
#                 callbacks=[checkpointer]
                )
encoder3.save(base_dir + '/ae' + model_version + '.h5')


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 31, 31, 1)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 27, 27, 32)        9248      
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 25, 25, 32)        9248      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 23, 23, 32)        9248      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 16)          0   

Epoch 67/500
35/35 [==============================] - 1s 16ms/step - loss: 8.9754e-04 - val_loss: 0.0343
Epoch 68/500
35/35 [==============================] - 1s 16ms/step - loss: 8.4074e-04 - val_loss: 0.0344
Epoch 69/500
35/35 [==============================] - 1s 16ms/step - loss: 8.2035e-04 - val_loss: 0.0346
Epoch 70/500
35/35 [==============================] - 1s 16ms/step - loss: 7.6102e-04 - val_loss: 0.0348
Epoch 71/500
35/35 [==============================] - 1s 16ms/step - loss: 7.4897e-04 - val_loss: 0.0350
Epoch 72/500
35/35 [==============================] - 1s 16ms/step - loss: 6.9638e-04 - val_loss: 0.0351
Epoch 73/500
35/35 [==============================] - 1s 16ms/step - loss: 6.7796e-04 - val_loss: 0.0353
Epoch 74/500
35/35 [==============================] - 1s 16ms/step - loss: 6.3363e-04 - val_loss: 0.0355
Epoch 75/500
35/35 [==============================] - 1s 16ms/step - loss: 6.1894e-04 - val_loss: 0.0356
Epoch 76/500
35/35 [==============================] - 1

35/35 [==============================] - 1s 16ms/step - loss: 3.6111e-05 - val_loss: 0.0407
Epoch 145/500
35/35 [==============================] - 1s 16ms/step - loss: 3.5354e-05 - val_loss: 0.0408
Epoch 146/500
35/35 [==============================] - 1s 16ms/step - loss: 3.2703e-05 - val_loss: 0.0408
Epoch 147/500
35/35 [==============================] - 1s 16ms/step - loss: 3.2410e-05 - val_loss: 0.0409
Epoch 148/500
35/35 [==============================] - 1s 15ms/step - loss: 3.0319e-05 - val_loss: 0.0408
Epoch 149/500
35/35 [==============================] - 1s 15ms/step - loss: 3.0461e-05 - val_loss: 0.0409
Epoch 150/500
35/35 [==============================] - 1s 16ms/step - loss: 2.8761e-05 - val_loss: 0.0409
Epoch 151/500
35/35 [==============================] - 1s 16ms/step - loss: 2.9121e-05 - val_loss: 0.0409
Epoch 152/500
35/35 [==============================] - 1s 16ms/step - loss: 2.7724e-05 - val_loss: 0.0410
Epoch 153/500
35/35 [==============================] - 1s 16

35/35 [==============================] - 1s 16ms/step - loss: 2.6160e-06 - val_loss: 0.0417
Epoch 222/500
35/35 [==============================] - 1s 16ms/step - loss: 2.5408e-06 - val_loss: 0.0417
Epoch 223/500
35/35 [==============================] - 1s 16ms/step - loss: 2.6476e-06 - val_loss: 0.0418
Epoch 224/500
35/35 [==============================] - 1s 16ms/step - loss: 2.6171e-06 - val_loss: 0.0417
Epoch 225/500
35/35 [==============================] - 1s 16ms/step - loss: 2.6245e-06 - val_loss: 0.0417
Epoch 226/500
35/35 [==============================] - 1s 17ms/step - loss: 2.4242e-06 - val_loss: 0.0417
Epoch 227/500
35/35 [==============================] - 1s 16ms/step - loss: 2.2983e-06 - val_loss: 0.0417
Epoch 228/500
35/35 [==============================] - 1s 16ms/step - loss: 2.0868e-06 - val_loss: 0.0418
Epoch 229/500
35/35 [==============================] - 1s 16ms/step - loss: 2.0008e-06 - val_loss: 0.0418
Epoch 230/500
35/35 [==============================] - 1s 16

35/35 [==============================] - 1s 16ms/step - loss: 1.2476e-07 - val_loss: 0.0419
Epoch 299/500
35/35 [==============================] - 1s 16ms/step - loss: 1.3803e-07 - val_loss: 0.0419
Epoch 300/500
35/35 [==============================] - 1s 16ms/step - loss: 1.4585e-07 - val_loss: 0.0419
Epoch 301/500
35/35 [==============================] - 1s 16ms/step - loss: 1.6391e-07 - val_loss: 0.0419
Epoch 302/500
35/35 [==============================] - 1s 16ms/step - loss: 1.7947e-07 - val_loss: 0.0419
Epoch 303/500
35/35 [==============================] - 1s 15ms/step - loss: 2.0593e-07 - val_loss: 0.0419
Epoch 304/500
35/35 [==============================] - 1s 16ms/step - loss: 2.2852e-07 - val_loss: 0.0419
Epoch 305/500
35/35 [==============================] - 1s 16ms/step - loss: 2.5754e-07 - val_loss: 0.0419
Epoch 306/500
35/35 [==============================] - 1s 15ms/step - loss: 2.7730e-07 - val_loss: 0.0419
Epoch 307/500
35/35 [==============================] - 1s 16

35/35 [==============================] - 1s 16ms/step - loss: 1.3914e-08 - val_loss: 0.0419
Epoch 376/500
35/35 [==============================] - 1s 15ms/step - loss: 1.4403e-08 - val_loss: 0.0419
Epoch 377/500
35/35 [==============================] - 1s 16ms/step - loss: 1.5393e-08 - val_loss: 0.0419
Epoch 378/500
35/35 [==============================] - 1s 16ms/step - loss: 1.5951e-08 - val_loss: 0.0419
Epoch 379/500
35/35 [==============================] - 1s 15ms/step - loss: 1.7109e-08 - val_loss: 0.0419
Epoch 380/500
35/35 [==============================] - 1s 16ms/step - loss: 1.7970e-08 - val_loss: 0.0419
Epoch 381/500
35/35 [==============================] - 1s 15ms/step - loss: 1.9306e-08 - val_loss: 0.0419
Epoch 382/500
35/35 [==============================] - 1s 16ms/step - loss: 2.0122e-08 - val_loss: 0.0419
Epoch 383/500
35/35 [==============================] - 1s 16ms/step - loss: 2.1185e-08 - val_loss: 0.0419
Epoch 384/500
35/35 [==============================] - 1s 16

35/35 [==============================] - 1s 16ms/step - loss: 1.6504e-08 - val_loss: 0.0419
Epoch 453/500
35/35 [==============================] - 1s 16ms/step - loss: 1.5676e-08 - val_loss: 0.0419
Epoch 454/500
35/35 [==============================] - 1s 16ms/step - loss: 1.4781e-08 - val_loss: 0.0419
Epoch 455/500
35/35 [==============================] - 1s 16ms/step - loss: 1.3753e-08 - val_loss: 0.0419
Epoch 456/500
35/35 [==============================] - 1s 16ms/step - loss: 1.2546e-08 - val_loss: 0.0419
Epoch 457/500
35/35 [==============================] - 1s 16ms/step - loss: 1.1439e-08 - val_loss: 0.0419
Epoch 458/500
35/35 [==============================] - 1s 16ms/step - loss: 1.0428e-08 - val_loss: 0.0419
Epoch 459/500
35/35 [==============================] - 1s 16ms/step - loss: 9.6200e-09 - val_loss: 0.0419
Epoch 460/500
35/35 [==============================] - 1s 16ms/step - loss: 8.8276e-09 - val_loss: 0.0419
Epoch 461/500
35/35 [==============================] - 1s 16

In [14]:
input_shape = (img_width, img_height, 1)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="selu", padding="valid")(input_img)
x = Conv2D(32, (3, 3), activation="selu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="selu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="selu", padding="valid")(x)
x = Conv2D(16, (3, 3), activation="selu", padding="valid")(x)
x = MaxPooling2D((4, 4), padding="valid")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(129, activation="sigmoid")(x)

encoder4 = Model(input_img, encoded)

encoder4.compile(optimizer='adadelta', loss="binary_crossentropy")
encoder4.summary()

model_version = '2.0.0.0.11_encoder_selueverywhere_sigmoidlast_bce_adadelta_ridiculouslysmall'

checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder4.fit_generator(fixed_generator(x_train, y_train, batch_size),
                steps_per_epoch=x_train.shape[0],
                epochs=500,
                validation_data=fixed_generator(x_validation, y_validation, batch_size),
                validation_steps=x_validation.shape[0]#,
#                 callbacks=[checkpointer]
                )
encoder4.save(base_dir + '/ae' + model_version + '.h5')


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 31, 31, 1)         0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 27, 27, 32)        9248      
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 25, 25, 32)        9248      
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 23, 23, 32)        9248      
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 16)          0   

Epoch 67/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2490 - val_loss: 0.4757
Epoch 68/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2503 - val_loss: 0.4773
Epoch 69/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2489 - val_loss: 0.4728
Epoch 70/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2501 - val_loss: 0.4686
Epoch 71/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2489 - val_loss: 0.4692
Epoch 72/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2501 - val_loss: 0.4685
Epoch 73/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2488 - val_loss: 0.4716
Epoch 74/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2501 - val_loss: 0.4738
Epoch 75/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2488 - val_loss: 0.4766
Epoch 76/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2500 - val_los

35/35 [==============================] - 1s 23ms/step - loss: 0.2483 - val_loss: 0.5020
Epoch 148/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2495 - val_loss: 0.4953
Epoch 149/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2483 - val_loss: 0.5035
Epoch 150/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2495 - val_loss: 0.4989
Epoch 151/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2482 - val_loss: 0.5025
Epoch 152/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2495 - val_loss: 0.5052
Epoch 153/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2482 - val_loss: 0.5006
Epoch 154/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2495 - val_loss: 0.5075
Epoch 155/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2482 - val_loss: 0.5007
Epoch 156/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2495 - val_loss: 0

35/35 [==============================] - 1s 23ms/step - loss: 0.2481 - val_loss: 0.5205
Epoch 228/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2494 - val_loss: 0.5212
Epoch 229/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2481 - val_loss: 0.5176
Epoch 230/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2494 - val_loss: 0.5240
Epoch 231/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2481 - val_loss: 0.5200
Epoch 232/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2494 - val_loss: 0.5147
Epoch 233/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2481 - val_loss: 0.5212
Epoch 234/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2494 - val_loss: 0.5239
Epoch 235/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2481 - val_loss: 0.5191
Epoch 236/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2494 - val_loss: 0

35/35 [==============================] - 1s 23ms/step - loss: 0.2480 - val_loss: 0.5345
Epoch 308/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2493 - val_loss: 0.5371
Epoch 309/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2480 - val_loss: 0.5354
Epoch 310/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2493 - val_loss: 0.5358
Epoch 311/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2480 - val_loss: 0.5340
Epoch 312/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2493 - val_loss: 0.5318
Epoch 313/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2480 - val_loss: 0.5331
Epoch 314/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2493 - val_loss: 0.5322
Epoch 315/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2480 - val_loss: 0.5340
Epoch 316/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2493 - val_loss: 0

35/35 [==============================] - 1s 23ms/step - loss: 0.2480 - val_loss: 0.5439
Epoch 388/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2493 - val_loss: 0.5466
Epoch 389/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2480 - val_loss: 0.5454
Epoch 390/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2493 - val_loss: 0.5473
Epoch 391/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2480 - val_loss: 0.5455
Epoch 392/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2493 - val_loss: 0.5478
Epoch 393/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2480 - val_loss: 0.5445
Epoch 394/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2493 - val_loss: 0.5451
Epoch 395/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2480 - val_loss: 0.5452
Epoch 396/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2493 - val_loss: 0

35/35 [==============================] - 1s 23ms/step - loss: 0.2480 - val_loss: 0.5541
Epoch 468/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2493 - val_loss: 0.5558
Epoch 469/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2480 - val_loss: 0.5532
Epoch 470/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2493 - val_loss: 0.5547
Epoch 471/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2480 - val_loss: 0.5538
Epoch 472/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2493 - val_loss: 0.5554
Epoch 473/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2480 - val_loss: 0.5527
Epoch 474/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2493 - val_loss: 0.5525
Epoch 475/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2480 - val_loss: 0.5530
Epoch 476/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2493 - val_loss: 0

In [15]:
input_shape = (img_width, img_height, 1)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="selu", padding="valid")(input_img)
x = Conv2D(32, (3, 3), activation="selu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="selu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="selu", padding="valid")(x)
x = Conv2D(16, (3, 3), activation="selu", padding="valid")(x)
x = MaxPooling2D((4, 4), padding="valid")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(129, activation="elu")(x)

encoder5 = Model(input_img, encoded)

encoder5.compile(optimizer='adadelta', loss="binary_crossentropy")
encoder5.summary()

model_version = '2.0.0.0.12_encoder_selueverywhere_bce_adadelta_ridiculouslysmall'

checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder5.fit_generator(fixed_generator(x_train, y_train, batch_size),
                steps_per_epoch=x_train.shape[0],
                epochs=500,
                validation_data=fixed_generator(x_validation, y_validation, batch_size),
                validation_steps=x_validation.shape[0]#,
#                 callbacks=[checkpointer]
                )
encoder5.save(base_dir + '/ae' + model_version + '.h5')


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 31, 31, 1)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 27, 27, 32)        9248      
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 25, 25, 32)        9248      
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 23, 23, 32)        9248      
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 16)          0   

Epoch 67/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2865 - val_loss: 0.6133
Epoch 68/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2882 - val_loss: 0.6113
Epoch 69/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2855 - val_loss: 0.6190
Epoch 70/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2990 - val_loss: 0.6331
Epoch 71/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2891 - val_loss: 0.6206
Epoch 72/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2968 - val_loss: 0.6093
Epoch 73/500
35/35 [==============================] - 1s 23ms/step - loss: 0.3022 - val_loss: 0.6480
Epoch 74/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2952 - val_loss: 0.6386
Epoch 75/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2876 - val_loss: 0.6544
Epoch 76/500
35/35 [==============================] - 1s 24ms/step - loss: 0.2881 - val_los

35/35 [==============================] - 1s 23ms/step - loss: 0.2748 - val_loss: 0.6008
Epoch 148/500
35/35 [==============================] - 1s 24ms/step - loss: 0.2824 - val_loss: 0.5827
Epoch 149/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2836 - val_loss: 0.5988
Epoch 150/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2801 - val_loss: 0.6052
Epoch 151/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2765 - val_loss: 0.6282
Epoch 152/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2756 - val_loss: 0.6114
Epoch 153/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2795 - val_loss: 0.6013
Epoch 154/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2877 - val_loss: 0.6062
Epoch 155/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2768 - val_loss: 0.6086
Epoch 156/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2844 - val_loss: 0

35/35 [==============================] - 1s 23ms/step - loss: 0.2659 - val_loss: 0.5918
Epoch 228/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2675 - val_loss: 0.5780
Epoch 229/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2741 - val_loss: 0.5672
Epoch 230/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2740 - val_loss: 0.5910
Epoch 231/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2684 - val_loss: 0.6007
Epoch 232/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2665 - val_loss: 0.6120
Epoch 233/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2649 - val_loss: 0.6125
Epoch 234/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2686 - val_loss: 0.6087
Epoch 235/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2690 - val_loss: 0.6132
Epoch 236/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2670 - val_loss: 0

35/35 [==============================] - 1s 23ms/step - loss: 0.2636 - val_loss: 0.5944
Epoch 308/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2642 - val_loss: 0.5555
Epoch 309/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2682 - val_loss: 0.5687
Epoch 310/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2668 - val_loss: 0.5398
Epoch 311/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2637 - val_loss: 0.5859
Epoch 312/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2658 - val_loss: 0.5575
Epoch 313/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2678 - val_loss: 0.5814
Epoch 314/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2649 - val_loss: 0.5678
Epoch 315/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2655 - val_loss: 0.5608
Epoch 316/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2655 - val_loss: 0

35/35 [==============================] - 1s 23ms/step - loss: 0.2632 - val_loss: 0.5930
Epoch 388/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2649 - val_loss: 0.5854
Epoch 389/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2637 - val_loss: 0.5807
Epoch 390/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2675 - val_loss: 0.5738
Epoch 391/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2639 - val_loss: 0.5845
Epoch 392/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2634 - val_loss: 0.5792
Epoch 393/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2651 - val_loss: 0.5568
Epoch 394/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2650 - val_loss: 0.5684
Epoch 395/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2629 - val_loss: 0.5719
Epoch 396/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2658 - val_loss: 0

35/35 [==============================] - 1s 23ms/step - loss: 0.2636 - val_loss: 0.5758
Epoch 468/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2647 - val_loss: 0.5817
Epoch 469/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2644 - val_loss: 0.5702
Epoch 470/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2638 - val_loss: 0.5782
Epoch 471/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2628 - val_loss: 0.5379
Epoch 472/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2686 - val_loss: 0.5490
Epoch 473/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2643 - val_loss: 0.5573
Epoch 474/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2630 - val_loss: 0.5643
Epoch 475/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2656 - val_loss: 0.5603
Epoch 476/500
35/35 [==============================] - 1s 23ms/step - loss: 0.2654 - val_loss: 0

In [16]:
input_shape = (img_width, img_height, 1)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="selu", padding="valid")(input_img)
x = Conv2D(32, (3, 3), activation="selu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="selu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="selu", padding="valid")(x)
x = Conv2D(16, (3, 3), activation="selu", padding="valid")(x)
x = MaxPooling2D((4, 4), padding="valid")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(129, activation="elu")(x)

encoder6 = Model(input_img, encoded)

encoder6.compile(optimizer='adadelta', loss="mse")
encoder6.summary()

model_version = '2.0.0.0.13_encoder_selueverywhere_mse_adadelta_ridiculouslysmall'

checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder6.fit_generator(fixed_generator(x_train, y_train, batch_size),
                steps_per_epoch=x_train.shape[0],
                epochs=500,
                validation_data=fixed_generator(x_validation, y_validation, batch_size),
                validation_steps=x_validation.shape[0]#,
#                 callbacks=[checkpointer]
                )
encoder6.save(base_dir + '/ae' + model_version + '.h5')


Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 31, 31, 1)         0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 27, 27, 32)        9248      
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 25, 25, 32)        9248      
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 23, 23, 32)        9248      
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 16)          0   

35/35 [==============================] - 1s 23ms/step - loss: 3.0588e-04 - val_loss: 0.0355
Epoch 67/500
35/35 [==============================] - 1s 23ms/step - loss: 2.8873e-04 - val_loss: 0.0349
Epoch 68/500
35/35 [==============================] - 1s 23ms/step - loss: 2.6547e-04 - val_loss: 0.0352
Epoch 69/500
35/35 [==============================] - 1s 23ms/step - loss: 2.6936e-04 - val_loss: 0.0352
Epoch 70/500
35/35 [==============================] - 1s 23ms/step - loss: 2.5206e-04 - val_loss: 0.0355
Epoch 71/500
35/35 [==============================] - 1s 23ms/step - loss: 2.3395e-04 - val_loss: 0.0357
Epoch 72/500
35/35 [==============================] - 1s 23ms/step - loss: 2.3131e-04 - val_loss: 0.0357
Epoch 73/500
35/35 [==============================] - 1s 23ms/step - loss: 2.2274e-04 - val_loss: 0.0358
Epoch 74/500
35/35 [==============================] - 1s 23ms/step - loss: 1.9861e-04 - val_loss: 0.0358
Epoch 75/500
35/35 [==============================] - 1s 23ms/step -

35/35 [==============================] - 1s 23ms/step - loss: 1.2173e-05 - val_loss: 0.0370
Epoch 144/500
35/35 [==============================] - 1s 23ms/step - loss: 1.1818e-05 - val_loss: 0.0370
Epoch 145/500
35/35 [==============================] - 1s 23ms/step - loss: 9.7691e-06 - val_loss: 0.0370
Epoch 146/500
35/35 [==============================] - 1s 23ms/step - loss: 9.6799e-06 - val_loss: 0.0370
Epoch 147/500
35/35 [==============================] - 1s 23ms/step - loss: 1.2569e-05 - val_loss: 0.0371
Epoch 148/500
35/35 [==============================] - 1s 23ms/step - loss: 1.3019e-05 - val_loss: 0.0371
Epoch 149/500
35/35 [==============================] - 1s 23ms/step - loss: 1.0739e-05 - val_loss: 0.0370
Epoch 150/500
35/35 [==============================] - 1s 23ms/step - loss: 1.0927e-05 - val_loss: 0.0370
Epoch 151/500
35/35 [==============================] - 1s 23ms/step - loss: 9.5841e-06 - val_loss: 0.0371
Epoch 152/500
35/35 [==============================] - 1s 23

35/35 [==============================] - 1s 23ms/step - loss: 9.1017e-07 - val_loss: 0.0371
Epoch 221/500
35/35 [==============================] - 1s 23ms/step - loss: 7.9038e-07 - val_loss: 0.0371
Epoch 222/500
35/35 [==============================] - 1s 23ms/step - loss: 7.4935e-07 - val_loss: 0.0371
Epoch 223/500
35/35 [==============================] - 1s 23ms/step - loss: 7.1412e-07 - val_loss: 0.0371
Epoch 224/500
35/35 [==============================] - 1s 23ms/step - loss: 7.1038e-07 - val_loss: 0.0371
Epoch 225/500
35/35 [==============================] - 1s 23ms/step - loss: 6.8159e-07 - val_loss: 0.0371
Epoch 226/500
35/35 [==============================] - 1s 24ms/step - loss: 5.4919e-07 - val_loss: 0.0371
Epoch 227/500
35/35 [==============================] - 1s 23ms/step - loss: 5.1639e-07 - val_loss: 0.0371
Epoch 228/500
35/35 [==============================] - 1s 23ms/step - loss: 5.9442e-07 - val_loss: 0.0371
Epoch 229/500
35/35 [==============================] - 1s 23

35/35 [==============================] - 1s 23ms/step - loss: 3.3557e-08 - val_loss: 0.0371
Epoch 298/500
35/35 [==============================] - 1s 23ms/step - loss: 3.8322e-08 - val_loss: 0.0371
Epoch 299/500
35/35 [==============================] - 1s 23ms/step - loss: 3.7310e-08 - val_loss: 0.0371
Epoch 300/500
35/35 [==============================] - 1s 23ms/step - loss: 3.1722e-08 - val_loss: 0.0371
Epoch 301/500
35/35 [==============================] - 1s 23ms/step - loss: 2.7310e-08 - val_loss: 0.0371
Epoch 302/500
35/35 [==============================] - 1s 23ms/step - loss: 2.3183e-08 - val_loss: 0.0371
Epoch 303/500
35/35 [==============================] - 1s 23ms/step - loss: 1.8471e-08 - val_loss: 0.0371
Epoch 304/500
35/35 [==============================] - 1s 23ms/step - loss: 1.5977e-08 - val_loss: 0.0371
Epoch 305/500
35/35 [==============================] - 1s 23ms/step - loss: 1.5716e-08 - val_loss: 0.0371
Epoch 306/500
35/35 [==============================] - 1s 23

35/35 [==============================] - 1s 23ms/step - loss: 1.2584e-09 - val_loss: 0.0371
Epoch 375/500
35/35 [==============================] - 1s 23ms/step - loss: 1.4603e-09 - val_loss: 0.0371
Epoch 376/500
35/35 [==============================] - 1s 23ms/step - loss: 1.3203e-09 - val_loss: 0.0371
Epoch 377/500
35/35 [==============================] - 1s 23ms/step - loss: 1.0180e-09 - val_loss: 0.0371
Epoch 378/500
35/35 [==============================] - 1s 23ms/step - loss: 7.7684e-10 - val_loss: 0.0371
Epoch 379/500
35/35 [==============================] - 1s 23ms/step - loss: 6.6324e-10 - val_loss: 0.0371
Epoch 380/500
35/35 [==============================] - 1s 23ms/step - loss: 6.2428e-10 - val_loss: 0.0371
Epoch 381/500
35/35 [==============================] - 1s 23ms/step - loss: 6.5663e-10 - val_loss: 0.0371
Epoch 382/500
35/35 [==============================] - 1s 23ms/step - loss: 6.5543e-10 - val_loss: 0.0371
Epoch 383/500
35/35 [==============================] - 1s 23

35/35 [==============================] - 1s 23ms/step - loss: 9.0849e-11 - val_loss: 0.0371
Epoch 452/500
35/35 [==============================] - 1s 23ms/step - loss: 7.3784e-11 - val_loss: 0.0371
Epoch 453/500
35/35 [==============================] - 1s 23ms/step - loss: 5.9842e-11 - val_loss: 0.0371
Epoch 454/500
35/35 [==============================] - 1s 23ms/step - loss: 6.1510e-11 - val_loss: 0.0371
Epoch 455/500
35/35 [==============================] - 1s 23ms/step - loss: 7.1725e-11 - val_loss: 0.0371
Epoch 456/500
35/35 [==============================] - 1s 23ms/step - loss: 9.6018e-11 - val_loss: 0.0371
Epoch 457/500
35/35 [==============================] - 1s 23ms/step - loss: 1.0326e-10 - val_loss: 0.0371
Epoch 458/500
35/35 [==============================] - 1s 23ms/step - loss: 1.0874e-10 - val_loss: 0.0371
Epoch 459/500
35/35 [==============================] - 1s 23ms/step - loss: 9.7022e-11 - val_loss: 0.0371
Epoch 460/500
35/35 [==============================] - 1s 23

In [71]:
input_shape = (img_width, img_height, 1)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="relu", padding="valid")(input_img)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(16, (3, 3), activation="relu", padding="valid")(x)
x = MaxPooling2D((4, 4), padding="valid")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(129, activation="relu")(x)

encoder7 = Model(input_img, encoded)

encoder7.compile(optimizer='adadelta', loss="binary_crossentropy")
encoder7.summary()

model_version = '2.0.0.0.14_encoder_relueverywhere_bce_adadelta_ridiculouslysmall'

checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder7.fit_generator(fixed_generator(x_train, y_train, batch_size),
                steps_per_epoch=x_train.shape[0],
                epochs=500,
                validation_data=fixed_generator(x_validation, y_validation, batch_size),
                validation_steps=x_validation.shape[0]#,
#                 callbacks=[checkpointer]
                )
encoder7.save(base_dir + '/ae' + model_version + '.h5')


Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 31, 31, 1)         0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 27, 27, 32)        9248      
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 25, 25, 32)        9248      
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 23, 23, 32)        9248      
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 16)          0   

Epoch 67/500
35/35 [==============================] - 1s 16ms/step - loss: 0.3204 - val_loss: 0.6519
Epoch 68/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3243 - val_loss: 0.6049
Epoch 69/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3190 - val_loss: 0.6537
Epoch 70/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3208 - val_loss: 0.6228
Epoch 71/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3188 - val_loss: 0.6636
Epoch 72/500
35/35 [==============================] - 1s 16ms/step - loss: 0.3207 - val_loss: 0.6548
Epoch 73/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3212 - val_loss: 0.6568
Epoch 74/500
35/35 [==============================] - 1s 16ms/step - loss: 0.3205 - val_loss: 0.6640
Epoch 75/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3190 - val_loss: 0.6454
Epoch 76/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3234 - val_los

35/35 [==============================] - 1s 15ms/step - loss: 0.3125 - val_loss: 0.6179
Epoch 148/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3097 - val_loss: 0.6345
Epoch 149/500
35/35 [==============================] - 1s 16ms/step - loss: 0.3113 - val_loss: 0.6290
Epoch 150/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3126 - val_loss: 0.6333
Epoch 151/500
35/35 [==============================] - 1s 16ms/step - loss: 0.3109 - val_loss: 0.6362
Epoch 152/500
35/35 [==============================] - 1s 16ms/step - loss: 0.3098 - val_loss: 0.6483
Epoch 153/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3130 - val_loss: 0.5938
Epoch 154/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3128 - val_loss: 0.6431
Epoch 155/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3088 - val_loss: 0.6375
Epoch 156/500
35/35 [==============================] - 1s 16ms/step - loss: 0.3108 - val_loss: 0

35/35 [==============================] - 1s 16ms/step - loss: 0.3013 - val_loss: 0.6054
Epoch 228/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2999 - val_loss: 0.6353
Epoch 229/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2990 - val_loss: 0.5653
Epoch 230/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3001 - val_loss: 0.5915
Epoch 231/500
35/35 [==============================] - 1s 16ms/step - loss: 0.3001 - val_loss: 0.5786
Epoch 232/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3070 - val_loss: 0.5761
Epoch 233/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3000 - val_loss: 0.6183
Epoch 234/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2996 - val_loss: 0.6240
Epoch 235/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3006 - val_loss: 0.5396
Epoch 236/500
35/35 [==============================] - 1s 15ms/step - loss: 0.3061 - val_loss: 0

35/35 [==============================] - 1s 16ms/step - loss: 0.2949 - val_loss: 0.5842
Epoch 308/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2946 - val_loss: 0.5577
Epoch 309/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2963 - val_loss: 0.5560
Epoch 310/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2938 - val_loss: 0.5586
Epoch 311/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2930 - val_loss: 0.5774
Epoch 312/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2938 - val_loss: 0.5853
Epoch 313/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2928 - val_loss: 0.5451
Epoch 314/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2930 - val_loss: 0.5867
Epoch 315/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2940 - val_loss: 0.5642
Epoch 316/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2961 - val_loss: 0

35/35 [==============================] - 1s 16ms/step - loss: 0.2914 - val_loss: 0.4856
Epoch 388/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2945 - val_loss: 0.5353
Epoch 389/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2920 - val_loss: 0.5224
Epoch 390/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2929 - val_loss: 0.5775
Epoch 391/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2912 - val_loss: 0.5477
Epoch 392/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2929 - val_loss: 0.5306
Epoch 393/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2920 - val_loss: 0.5497
Epoch 394/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2925 - val_loss: 0.5326
Epoch 395/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2939 - val_loss: 0.5663
Epoch 396/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2925 - val_loss: 0

35/35 [==============================] - 1s 15ms/step - loss: 0.2915 - val_loss: 0.5431
Epoch 468/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2925 - val_loss: 0.5137
Epoch 469/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2969 - val_loss: 0.5458
Epoch 470/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2937 - val_loss: 0.5518
Epoch 471/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2918 - val_loss: 0.5474
Epoch 472/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2933 - val_loss: 0.5417
Epoch 473/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2907 - val_loss: 0.5696
Epoch 474/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2938 - val_loss: 0.5529
Epoch 475/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2923 - val_loss: 0.5430
Epoch 476/500
35/35 [==============================] - 1s 15ms/step - loss: 0.2951 - val_loss: 0

In [72]:
input_shape = (img_width, img_height, 1)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="relu", padding="valid")(input_img)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(16, (3, 3), activation="relu", padding="valid")(x)
x = MaxPooling2D((4, 4), padding="valid")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(129, activation="relu")(x)

encoder8 = Model(input_img, encoded)

encoder8.compile(optimizer='adadelta', loss="mse")
encoder8.summary()

model_version = '2.0.0.0.15_encoder_relueverywhere_mse_adadelta_ridiculouslysmall'

checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder8.fit_generator(fixed_generator(x_train, y_train, batch_size),
                steps_per_epoch=x_train.shape[0],
                epochs=500,
                validation_data=fixed_generator(x_validation, y_validation, batch_size),
                validation_steps=x_validation.shape[0]#,
#                 callbacks=[checkpointer]
                )
encoder8.save(base_dir + '/ae' + model_version + '.h5')


Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 31, 31, 1)         0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 27, 27, 32)        9248      
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 25, 25, 32)        9248      
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 23, 23, 32)        9248      
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 5, 5, 16)          0   

Epoch 67/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0062 - val_loss: 0.0325
Epoch 68/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0062 - val_loss: 0.0310
Epoch 69/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0061 - val_loss: 0.0332
Epoch 70/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0061 - val_loss: 0.0319
Epoch 71/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0060 - val_loss: 0.0316
Epoch 72/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0060 - val_loss: 0.0327
Epoch 73/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0059 - val_loss: 0.0326
Epoch 74/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0059 - val_loss: 0.0336
Epoch 75/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0058 - val_loss: 0.0316
Epoch 76/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0058 - val_los

35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0335
Epoch 148/500
35/35 [==============================] - 1s 16ms/step - loss: 0.0053 - val_loss: 0.0330
Epoch 149/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0338
Epoch 150/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0053 - val_loss: 0.0333
Epoch 151/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 152/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0053 - val_loss: 0.0337
Epoch 153/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0332
Epoch 154/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0053 - val_loss: 0.0333
Epoch 155/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0335
Epoch 156/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0053 - val_loss: 0

35/35 [==============================] - 1s 16ms/step - loss: 0.0052 - val_loss: 0.0334
Epoch 228/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0332
Epoch 229/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0334
Epoch 230/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 231/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 232/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0334
Epoch 233/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0334
Epoch 234/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 235/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0334
Epoch 236/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0

35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0334
Epoch 308/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 309/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0334
Epoch 310/500
35/35 [==============================] - 1s 16ms/step - loss: 0.0052 - val_loss: 0.0334
Epoch 311/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 312/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0334
Epoch 313/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 314/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 315/500
35/35 [==============================] - 1s 16ms/step - loss: 0.0052 - val_loss: 0.0334
Epoch 316/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0

35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 388/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0334
Epoch 389/500
35/35 [==============================] - 1s 16ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 390/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 391/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 392/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 393/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 394/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 395/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 396/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0

35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 468/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 469/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 470/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 471/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 472/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 473/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 474/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 475/500
35/35 [==============================] - 1s 16ms/step - loss: 0.0052 - val_loss: 0.0333
Epoch 476/500
35/35 [==============================] - 1s 15ms/step - loss: 0.0052 - val_loss: 0

In [81]:
input_shape = (img_width, img_height, 1)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="elu", padding="valid")(input_img)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(16, (3, 3), activation="elu", padding="valid")(x)
x = MaxPooling2D((4, 4), padding="valid")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(129, activation="elu", bias_initializer=initializers.Constant(0.58))(x)

encoder9 = Model(input_img, encoded)

encoder9.compile(optimizer='adadelta', loss="binary_crossentropy")
encoder9.summary()

model_version = '2.0.0.0.16_encoder_elueverywhere_bce_adadelta_initbias0.58_ridiculouslysmall'

checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder9.fit_generator(fixed_generator(x_train, y_train, batch_size),
                steps_per_epoch=x_train.shape[0],
                epochs=500,
                validation_data=fixed_generator(x_validation, y_validation, batch_size),
                validation_steps=x_validation.shape[0]#,
#                 callbacks=[checkpointer]
                )
encoder9.save(base_dir + '/ae' + model_version + '.h5')


Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 31, 31, 1)         0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 27, 27, 32)        9248      
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 25, 25, 32)        9248      
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 23, 23, 32)        9248      
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 5, 5, 16)          0   

Epoch 67/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2710 - val_loss: 0.5920
Epoch 68/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2631 - val_loss: 0.5222
Epoch 69/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2657 - val_loss: 0.5821
Epoch 70/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2630 - val_loss: 0.5498
Epoch 71/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2670 - val_loss: 0.4928
Epoch 72/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2703 - val_loss: 0.5370
Epoch 73/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2641 - val_loss: 0.5915
Epoch 74/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2621 - val_loss: 0.5367
Epoch 75/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2627 - val_loss: 0.5916
Epoch 76/500
35/35 [==============================] - 1s 17ms/step - loss: 0.2612 - val_los

35/35 [==============================] - 1s 16ms/step - loss: 0.2614 - val_loss: 0.5864
Epoch 148/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2656 - val_loss: 0.5825
Epoch 149/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2621 - val_loss: 0.5921
Epoch 150/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2671 - val_loss: 0.5729
Epoch 151/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2578 - val_loss: 0.5584
Epoch 152/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2600 - val_loss: 0.5748
Epoch 153/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2599 - val_loss: 0.6178
Epoch 154/500
35/35 [==============================] - 1s 17ms/step - loss: 0.2627 - val_loss: 0.5631
Epoch 155/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2616 - val_loss: 0.6015
Epoch 156/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2582 - val_loss: 0

35/35 [==============================] - 1s 16ms/step - loss: 0.2578 - val_loss: 0.5597
Epoch 228/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2558 - val_loss: 0.5662
Epoch 229/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2530 - val_loss: 0.5261
Epoch 230/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2557 - val_loss: 0.5436
Epoch 231/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2543 - val_loss: 0.5452
Epoch 232/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2570 - val_loss: 0.5692
Epoch 233/500
35/35 [==============================] - 1s 17ms/step - loss: 0.2616 - val_loss: 0.5365
Epoch 234/500
35/35 [==============================] - 1s 17ms/step - loss: 0.2560 - val_loss: 0.5773
Epoch 235/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2540 - val_loss: 0.5619
Epoch 236/500
35/35 [==============================] - 1s 17ms/step - loss: 0.2565 - val_loss: 0

35/35 [==============================] - 1s 16ms/step - loss: 0.2538 - val_loss: 0.5318
Epoch 308/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2552 - val_loss: 0.5473
Epoch 309/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2542 - val_loss: 0.5005
Epoch 310/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2568 - val_loss: 0.5273
Epoch 311/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2562 - val_loss: 0.5303
Epoch 312/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2534 - val_loss: 0.5487
Epoch 313/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2546 - val_loss: 0.5266
Epoch 314/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2555 - val_loss: 0.5458
Epoch 315/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2517 - val_loss: 0.5448
Epoch 316/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2553 - val_loss: 0

35/35 [==============================] - 1s 16ms/step - loss: 0.2546 - val_loss: 0.5243
Epoch 388/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2532 - val_loss: 0.5169
Epoch 389/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2525 - val_loss: 0.4731
Epoch 390/500
35/35 [==============================] - 1s 17ms/step - loss: 0.2540 - val_loss: 0.5100
Epoch 391/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2568 - val_loss: 0.4656
Epoch 392/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2552 - val_loss: 0.5118
Epoch 393/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2523 - val_loss: 0.5264
Epoch 394/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2529 - val_loss: 0.5295
Epoch 395/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2533 - val_loss: 0.4932
Epoch 396/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2551 - val_loss: 0

35/35 [==============================] - 1s 16ms/step - loss: 0.2596 - val_loss: 0.5027
Epoch 468/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2554 - val_loss: 0.5226
Epoch 469/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2534 - val_loss: 0.5312
Epoch 470/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2585 - val_loss: 0.5077
Epoch 471/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2553 - val_loss: 0.4949
Epoch 472/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2559 - val_loss: 0.5291
Epoch 473/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2548 - val_loss: 0.5087
Epoch 474/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2597 - val_loss: 0.5126
Epoch 475/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2550 - val_loss: 0.5254
Epoch 476/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2544 - val_loss: 0

In [82]:
np.around(encoder9.predict(imageio.imread(train_data_dir + "/class0/patch000001.bmp").reshape(1,31,31,1))*255)

array([[-255., -215., -208., -219., 4126.,  727., -238., -255., -253.,
        -251., -229., -250., 1749.,  364., -206., -255., 2792.,  894.,
        -255., -255., -234., -254., -252., -252.,  511., -238., -253.,
        -241., 1800., -245., -255., -253., -255., -250., -149., -252.,
        3908., 1085., -253., -255., -255., -255., -242., -139., 2638.,
        -180., -254., -254., 2183.,  422., -242., -255., -254., -254.,
        -255., -253.,  369., -221., -226., 1878., 2020., -255., -255.,
        -255., -224., -216., -245., -249., 3017.,  -85., -255., -255.,
        -255., -255., -255., -254.,  536., 1404., -228., -239., 2630.,
        -245., -255., -255., -252.,  141., -248., -154., -255., -255.,
        -254., 2397., 1689., -183., -239., -255., 1459., -156., -179.,
        -253., 2025., -248., -254., -252., -161., -250., -225., -253.,
        -252., -244., -242., -244.,  308., -250., -250., -255., -205.,
        -229., -250.,  -23., -255., -255., -255., -223., 4070.,  250.,
      

In [79]:
np.around(encoder2.predict(imageio.imread(train_data_dir + "/class0/patch000001.bmp").reshape(1,31,31,1)))

array([[-1.,  2.,  1.,  0., 17.,  8., -1., -1., -1., -1., -1.,  0.,  8.,
         3.,  2., -1., 15.,  1., -1., -1.,  1., -1., -1., -1.,  5., -0.,
         1.,  2.,  4., -1., -1., -1., -1.,  1.,  0.,  1., 15.,  7., -1.,
        -1., -1., -1., -1., -1., 10.,  4., -1., -1.,  7., -0., -1., -0.,
         3., -1., -1., -1., -0.,  0.,  1.,  6.,  8., -1., -1., -1., -1.,
         3.,  2.,  2., 10.,  5., -1., -1., -1., -1., -1., -1.,  4.,  7.,
        -1., -0.,  7., -1., -1., -1.,  1.,  0., -1.,  3., -1., -1., -1.,
        11., 10., -1., -1., -1.,  3., -1., -1., -1.,  9.,  4., -1., -1.,
        -1., -1., -1., -1., -0.,  0., -1., -0.,  1., -1., -1., -1.,  1.,
        -0., -1.,  3., -1., -1., -1., -1., 17.,  4., -0., -1., -1.]],
      dtype=float32)

In [83]:
np.around(encoder9.predict(imageio.imread(train_data_dir + "/class0/patch000001.bmp").reshape(1,31,31,1)))

array([[-1., -1., -1., -1., 16.,  3., -1., -1., -1., -1., -1., -1.,  7.,
         1., -1., -1., 11.,  4., -1., -1., -1., -1., -1., -1.,  2., -1.,
        -1., -1.,  7., -1., -1., -1., -1., -1., -1., -1., 15.,  4., -1.,
        -1., -1., -1., -1., -1., 10., -1., -1., -1.,  9.,  2., -1., -1.,
        -1., -1., -1., -1.,  1., -1., -1.,  7.,  8., -1., -1., -1., -1.,
        -1., -1., -1., 12., -0., -1., -1., -1., -1., -1., -1.,  2.,  6.,
        -1., -1., 10., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1.,
         9.,  7., -1., -1., -1.,  6., -1., -1., -1.,  8., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1.,
        -1., -1., -0., -1., -1., -1., -1., 16.,  1., -1., -1., -1.]],
      dtype=float32)

In [23]:
np.load(train_descrs_dir + "/class0/descr_angle_000001.npy")

array([  4,   1,   0,   0, 137,  42,   1,   3,  55,   7,   0,   0,  71,
        32,   1,   3, 137,  39,   0,   0,   3,   2,   0,   5,  40,  10,
         1,  13,  81,   1,   0,   1,  11,   2,   0,   0, 137,  64,   1,
         1,  54,   2,   0,   0, 109,  23,   1,   4, 137,  33,   4,   2,
         8,   2,   0,   8,  40,  16,  12,  83,  83,   0,   0,   1,  27,
         9,   0,   1, 137,  58,   2,   0,  23,   0,   0,   0,  54,  62,
        17,  18, 137,  36,  12,   8,   5,  10,   9,  34,  10,  16,  22,
       137,  76,   2,   0,   0,  50,   8,   0,   1, 137,  57,   1,   1,
        26,   1,   0,   0,  31,  26,  18,  19,  19,   2,   4,   4,  10,
        10,  17,  47,   1,   1,   2,  27, 137,  25,   2,   3,  19],
      dtype=uint8)

In [84]:
encoders = [encoder1, encoder2, encoder3, encoder4, encoder5, encoder6, encoder7, encoder8, encoder9]
for encoder in encoders:
    corr_coefs = []
    for i in range(103):
        try:
            temp_y_p1 = np.load(train_descrs_dir + "/class0/descr_angle_" + str(i).zfill(6) + ".npy")
            temp_x_p1 = np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch" + str(i).zfill(6) + ".bmp").reshape(1,31,31,1))*255)
            temp_x_p1 = temp_x_p1.reshape(129)
            corr_coefs.append(np.corrcoef(temp_x_p1, temp_y_p1)[0,1])
        except:
            continue
    corr_coefs = np.array(corr_coefs)
    print(np.median(corr_coefs))

0.2878399491886138
0.7600350782532179
0.29238238343276507
-0.025160216073391452
0.7161978110298458
0.3056098336430682
0.7189368182917655
0.548305149989883
0.7949484586473158


In [67]:
input_shape = (img_width, img_height, 1)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="relu", padding="valid")(input_img)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(16, (3, 3), activation="relu", padding="valid")(x)
x = MaxPooling2D((4, 4), padding="valid")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(129, activation="tanh")(x)

encoder = Model(input_img, encoded)

encoder.compile(optimizer='adadelta', loss="binary_crossentropy")
#next up: encoder.compile(optimizer='sgd', metrics=['categorical_accuracy'], loss='categorical_crossentropy')


encoder.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 31, 31, 1)         0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 27, 27, 32)        9248      
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 25, 25, 32)        9248      
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 23, 23, 32)        9248      
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 5, 5, 16)          0   

In [68]:
model_version = '2.0.0.0.7_encoder_tanhlast_-0.2..0.8_bce_adadelta_ridiculouslysmall'

os.system('mkdir ' + base_dir + '/weights' + model_version)
checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder.fit_generator(fixed_generator(x_train, y_train, batch_size),
                steps_per_epoch=x_train.shape[0],
                epochs=500,
                validation_data=fixed_generator(x_validation, y_validation, batch_size),
                validation_steps=x_validation.shape[0]#,
#                 callbacks=[checkpointer]
                )
encoder.save(base_dir + '/ae' + model_version + '.h5')

# import keras.losses
# keras.losses.MSLE_plus_plus = MSLE_plus_plus
# encoder = load_model(base_dir + '/ae' + model_version + '.h5')

Epoch 1/500
35/35 [==============================] - 1s 21ms/step - loss: -1.3535 - val_loss: -1.3650
Epoch 2/500
35/35 [==============================] - 1s 15ms/step - loss: -1.4398 - val_loss: -1.3630
Epoch 3/500
35/35 [==============================] - 1s 15ms/step - loss: -1.4485 - val_loss: -1.3724
Epoch 4/500
35/35 [==============================] - 1s 15ms/step - loss: -1.4449 - val_loss: -1.3427
Epoch 5/500
35/35 [==============================] - 1s 15ms/step - loss: -1.4492 - val_loss: -1.3746
Epoch 6/500
35/35 [==============================] - 1s 15ms/step - loss: -1.4478 - val_loss: -1.3698
Epoch 7/500
35/35 [==============================] - 1s 15ms/step - loss: -1.4541 - val_loss: -1.3836
Epoch 8/500
35/35 [==============================] - 1s 16ms/step - loss: -1.4420 - val_loss: -1.3819
Epoch 9/500
35/35 [==============================] - 1s 15ms/step - loss: -1.4508 - val_loss: -1.3811
Epoch 10/500
35/35 [==============================] - 1s 16ms/step - loss: -1.4482

35/35 [==============================] - 1s 15ms/step - loss: -1.5679 - val_loss: -1.3871
Epoch 160/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5616 - val_loss: -1.3910
Epoch 161/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5685 - val_loss: -1.3907
Epoch 162/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5617 - val_loss: -1.3922
Epoch 163/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5683 - val_loss: -1.3926
Epoch 164/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5607 - val_loss: -1.3898
Epoch 165/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5682 - val_loss: -1.3877
Epoch 166/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5616 - val_loss: -1.3899
Epoch 167/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5684 - val_loss: -1.3916
Epoch 168/500
35/35 [==============================] - 1s 15ms/step - loss: -1

Epoch 238/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3873
Epoch 239/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3865
Epoch 240/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3865
Epoch 241/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3872
Epoch 242/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3864
Epoch 243/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3871
Epoch 244/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3868
Epoch 245/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3864
Epoch 246/500
35/35 [==============================] - 1s 16ms/step - loss: -1.5654 - val_loss: -1.3863
Epoch 247/500
35/35 [==============================] - 1s 15ms/s

Epoch 317/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3862
Epoch 318/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3863
Epoch 319/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3869
Epoch 320/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3862
Epoch 321/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3861
Epoch 322/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3864
Epoch 323/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3869
Epoch 324/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3861
Epoch 325/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3861
Epoch 326/500
35/35 [==============================] - 1s 15ms/s

Epoch 396/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3891
Epoch 397/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3881
Epoch 398/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3886
Epoch 399/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3890
Epoch 400/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3884
Epoch 401/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3887
Epoch 402/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3885
Epoch 403/500
35/35 [==============================] - 1s 16ms/step - loss: -1.5723 - val_loss: -1.3890
Epoch 404/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3891
Epoch 405/500
35/35 [==============================] - 1s 15ms/s

Epoch 475/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5740 - val_loss: -1.3876
Epoch 476/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5670 - val_loss: -1.3886
Epoch 477/500
35/35 [==============================] - 1s 16ms/step - loss: -1.5740 - val_loss: -1.3881
Epoch 478/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5670 - val_loss: -1.3887
Epoch 479/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5740 - val_loss: -1.3889
Epoch 480/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5670 - val_loss: -1.3885
Epoch 481/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5740 - val_loss: -1.3886
Epoch 482/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5670 - val_loss: -1.3885
Epoch 483/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5740 - val_loss: -1.3887
Epoch 484/500
35/35 [==============================] - 1s 15ms/s

In [21]:
np.set_printoptions(suppress=True)

In [69]:
np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000001.bmp").reshape(1,31,31,1))*255)

array([[-255., -255., -255., -255., -255., -255., -255., -255.,  255.,
        -255., -255., -255., -255., -255., -255., -255.,  255., -255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
        -255., -255., -255., -255.,  255., -255., -255., -255., -255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
        -255., -255., -255., -255., -255.,  255., -255., -255., -255.,
         255., -255., -255., -255., -255., -255., -255., -255.,  255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
        -255., -255., -255., -255.,  255., -255., -255., -255., -255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
      

In [46]:
np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000001.bmp").reshape(1,31,31,1))*255)

array([[ 46.,  23.,  14.,  11.,  20.,  15.,   8.,  20.,  80.,  46.,  20.,
         10.,  14.,  13.,  13.,  29.,  61.,  33.,  20.,  19.,  29.,  22.,
         19.,  27.,  20.,  20.,  11.,  24.,  40.,  22.,  25.,  18.,  42.,
         17.,  16.,  18.,  25.,  20.,  14.,  23.,  97.,  42.,  23.,  15.,
         19.,  16.,  14.,  33.,  85.,  43.,  17.,  22.,  36.,  28.,  27.,
         35.,  28.,  22.,  15.,  36.,  47.,  25.,  23.,  24.,  37.,  18.,
         11.,  12.,  27.,  35.,  28.,  30.,  90.,  28.,  14.,  15.,  23.,
         28.,  23.,  39.,  84.,  43.,  24.,  31.,  37.,  20.,  10.,  28.,
         27.,  23.,  27.,  37.,  42.,  20.,  12.,  21.,  28.,  11.,  13.,
         15.,  28.,  25.,  21.,  30.,  63.,  21.,  17.,  20.,  23.,  21.,
         18.,  42.,  50.,  23.,  19.,  28.,  33.,  19.,  16.,  35.,  25.,
         23.,  18.,  20.,  37.,  17.,  10.,  18., 147.]], dtype=float32)

In [22]:
np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000003.bmp").reshape(1,31,31,1))*255)

array([[  6226.,  -4327.,   -212.,  -5056.,  -7646.,  -6081.,   8609.,
          7355.,   5532.,   6354.,   5112.,   1746.,    469.,   4434.,
          5450.,   1333.,   5431.,  10231.,   6492.,     11.,   3948.,
          3639.,  -1945.,  -1682.,   -183.,  -2503.,   -992.,   -431.,
          4361.,   8124.,   2575.,  -1841.,   3022.,    -79.,   1061.,
         -3145.,   2384.,   -716.,   8723.,   2711.,   7590.,  -2574.,
          2260.,   -314.,   -263.,    203.,    861.,  -2148.,   4120.,
          1577.,   1211.,  -3280.,   2762.,   7455.,   6437.,   1561.,
           170.,  -4488.,   -589.,    465.,   2837.,   2627.,   3180.,
           319.,   1134.,   -845.,  -1492.,  -2196.,  -1676.,  12213.,
          -201.,   5783.,   7160.,   1915.,    378.,   4552.,   4211.,
           557.,   4883.,   4785.,   4315.,  -4458., -12434.,  -8737.,
          5939.,   8673.,   -182.,   -775.,   3257.,  -3857.,  -2258.,
         -3089.,  -1088.,  -2382.,  -1468.,   2597.,  -1078.,  -1202.,
      

In [45]:
np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000001.bmp").reshape(1,31,31,1))*255)

array([[ 46.,  23.,  14.,  11.,  20.,  15.,   8.,  20.,  80.,  46.,  20.,
         10.,  14.,  13.,  13.,  29.,  61.,  33.,  20.,  19.,  29.,  22.,
         19.,  27.,  20.,  20.,  11.,  24.,  40.,  22.,  25.,  18.,  42.,
         17.,  16.,  18.,  25.,  20.,  14.,  23.,  97.,  42.,  23.,  15.,
         19.,  16.,  14.,  33.,  85.,  43.,  17.,  22.,  36.,  28.,  27.,
         35.,  28.,  22.,  15.,  36.,  47.,  25.,  23.,  24.,  37.,  18.,
         11.,  12.,  27.,  35.,  28.,  30.,  90.,  28.,  14.,  15.,  23.,
         28.,  23.,  39.,  84.,  43.,  24.,  31.,  37.,  20.,  10.,  28.,
         27.,  23.,  27.,  37.,  42.,  20.,  12.,  21.,  28.,  11.,  13.,
         15.,  28.,  25.,  21.,  30.,  63.,  21.,  17.,  20.,  23.,  21.,
         18.,  42.,  50.,  23.,  19.,  28.,  33.,  19.,  16.,  35.,  25.,
         23.,  18.,  20.,  37.,  17.,  10.,  18., 147.]], dtype=float32)

In [30]:
np.load(train_descrs_dir + "/class0/descr_angle_000001.npy")

array([  4,   1,   0,   0, 137,  42,   1,   3,  55,   7,   0,   0,  71,
        32,   1,   3, 137,  39,   0,   0,   3,   2,   0,   5,  40,  10,
         1,  13,  81,   1,   0,   1,  11,   2,   0,   0, 137,  64,   1,
         1,  54,   2,   0,   0, 109,  23,   1,   4, 137,  33,   4,   2,
         8,   2,   0,   8,  40,  16,  12,  83,  83,   0,   0,   1,  27,
         9,   0,   1, 137,  58,   2,   0,  23,   0,   0,   0,  54,  62,
        17,  18, 137,  36,  12,   8,   5,  10,   9,  34,  10,  16,  22,
       137,  76,   2,   0,   0,  50,   8,   0,   1, 137,  57,   1,   1,
        26,   1,   0,   0,  31,  26,  18,  19,  19,   2,   4,   4,  10,
        10,  17,  47,   1,   1,   2,  27, 137,  25,   2,   3,  19],
      dtype=uint8)